In [1]:
using Revise
using BenchmarkTools

In [2]:
using DataIO

In [3]:
element_, score_ = eachcol(read_data("/Users/kwat/Downloads/score_and_set/ccle_proteinAHR/gene_x_score.tsv"))

;

In [4]:
using JSON

In [5]:
json_path = "/Users/kwat/Downloads/score_and_set/ccle_proteinAHR/gene_set.json"

json_dict = JSON.parse(open(json_path))

json_dict = convert(Dict{String, Vector{String}}, json_dict)

;

In [6]:
using GCTGMT

In [7]:
gmt_path_ = map(gmt_path -> replace(gmt_path, ".." => "/Users/kwat/Downloads/"), json_dict["gene_sets_tested"])

set_to_element_ = read_gmt(gmt_path_)

;

In [8]:
using FeatureSetEnrichment

┌ Info: Precompiling FeatureSetEnrichment [d7bef1b9-4643-4ed1-9ceb-ed43dd382ec0]
└ @ Base loading.jl:1317


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-1023812233257432316\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-16965853628518839389\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-1023812233257432316\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [ ]:
set_to_method_to_result = score_set_new(element_, score_, set_to_element_)

;

In [ ]:
values(set_to_method_to_result)

In [ ]:
set_ = collect(keys(set_to_element_))

n_set = length(set_)

method_ = collect(keys(collect(values(set_to_method_to_result))[1]))

In [ ]:
m = Matrix(undef, n_set, length(method_))

for (i, (set, method_to_result)) in enumerate(set_to_method_to_result)
    
    row = []
    
    for (method, result) in method_to_result
        
        append!(row, result[2])
        
    end
    
    m[i, :] = row  
    
end

In [ ]:
using Support

In [ ]:
for (ci, set_score_) in enumerate(eachcol(m))
    
    set_score_, set_ = sort_like((set_score_, set_))
    
    for set in json_dict["gene_sets_positive"]

        fi = findfirst(set_ .== set)

        println("$(method_[ci]) $fi")

    end

end